# Setup and Imports

In [1]:
# General imports
import numpy as np
import os

# TensorFlow/Keras imports
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# For deployment
from flask import Flask, request, jsonify
import tensorflow as tf

2024-11-20 14:38:50.287481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-20 14:38:50.287507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-20 14:38:50.288200: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 14:38:50.292043: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-20 14:38:50.752290: W tensorflow/compiler/tf2

# Model Loading and Tuning

In [3]:
# Load the pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(14, activation='softmax')(x)  # Adjust the number of classes

# Compile the final model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [7]:
import tensorflow as tf

# Check if any GPU is detected
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set TensorFlow to use only the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print(f"Using GPU: {gpus[0]}")
    except RuntimeError as e:
        print(e)

Num GPUs Available:  4
Visible devices cannot be modified after being initialized


In [8]:
# Paths to data directories
train_dir = '/home/exh4748/ProjectTortoise/split_data/train'
val_dir = '/home/exh4748/ProjectTortoise/split_data/val'

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

# Data loaders
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical'
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,  # Adjust as needed
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Save the model for later use
model.save('facial_recognition_model.h5')

Found 1445 images belonging to 14 classes.
Found 420 images belonging to 14 classes.
Epoch 1/100
45/45 [==============================] - 9s 189ms/step - loss: 1.3816 - accuracy: 0.5570 - val_loss: 0.9784 - val_accuracy: 0.6899
Epoch 2/100
45/45 [==============================] - 9s 190ms/step - loss: 1.2248 - accuracy: 0.6014 - val_loss: 0.8545 - val_accuracy: 0.7236
Epoch 3/100
45/45 [==============================] - 9s 189ms/step - loss: 1.1338 - accuracy: 0.6433 - val_loss: 0.8291 - val_accuracy: 0.7091
Epoch 4/100
45/45 [==============================] - 9s 190ms/step - loss: 1.0720 - accuracy: 0.6299 - val_loss: 0.7051 - val_accuracy: 0.7957
Epoch 5/100
45/45 [==============================] - 9s 188ms/step - loss: 0.9870 - accuracy: 0.6879 - val_loss: 0.6874 - val_accuracy: 0.7692
Epoch 6/100
45/45 [==============================] - 9s 188ms/step - loss: 0.9331 - accuracy: 0.6837 - val_loss: 0.6738 - val_accuracy: 0.7500
Epoch 7/100
45/45 [==============================] - 8s 1

45/45 [==============================] - 9s 188ms/step - loss: 0.3885 - accuracy: 0.8542 - val_loss: 0.3338 - val_accuracy: 0.8534
Epoch 58/100
45/45 [==============================] - 9s 190ms/step - loss: 0.3684 - accuracy: 0.8576 - val_loss: 0.3095 - val_accuracy: 0.8822
Epoch 59/100
45/45 [==============================] - 8s 184ms/step - loss: 0.3910 - accuracy: 0.8542 - val_loss: 0.3274 - val_accuracy: 0.8486
Epoch 60/100
45/45 [==============================] - 8s 187ms/step - loss: 0.3796 - accuracy: 0.8599 - val_loss: 0.3217 - val_accuracy: 0.8486
Epoch 61/100
45/45 [==============================] - 9s 188ms/step - loss: 0.3535 - accuracy: 0.8620 - val_loss: 0.3375 - val_accuracy: 0.8582
Epoch 62/100
45/45 [==============================] - 8s 187ms/step - loss: 0.3912 - accuracy: 0.8613 - val_loss: 0.3210 - val_accuracy: 0.8558
Epoch 63/100
45/45 [==============================] - 9s 191ms/step - loss: 0.3716 - accuracy: 0.8549 - val_loss: 0.3099 - val_accuracy: 0.8726
Epoch

/home/exh4748/.local/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Testing it on Efaz

In [24]:
# Test data directory
test_dir = '/home/exh4748/ProjectTortoise/split_data/test'

# Test data generator
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Predict all test samples
predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=1)

# Map indices to labels
class_labels = list(test_generator.class_indices.keys())
predicted_labels = [class_labels[idx] for idx in predicted_classes]

# Print sample predictions
print("Predicted Labels:", predicted_labels[:5])

Found 427 images belonging to 14 classes.
13/13 [==============================] - 0s 32ms/step - loss: 0.2828 - accuracy: 0.8558
Test Loss: 0.2828
Test Accuracy: 85.58%
14/14 [==============================] - 1s 34ms/step
Predicted Labels: ['ahmed', 'ahmed', 'ahmed', 'ahmed', 'ahmed']


# Inference

In [18]:
import random
# Load the saved model
model = load_model('facial_recognition_model.h5')

# Function to preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Adjust to model's input size
    img_array = img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function for prediction
def predict(image_path):
    processed_image = preprocess_image(image_path)
    predictions = model.predict(processed_image)
    class_index = np.argmax(predictions)
    confidence = np.max(predictions)
    return class_index, confidence

# Randomly pick and predict images from the test directory
def predict_random_images(test_dir, num_images=5):
    # Collect all image paths from test directory
    all_images = []
    for root, _, files in os.walk(test_dir):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                all_images.append(os.path.join(root, file))
    
    # Randomly select images to predict
    selected_images = random.sample(all_images, min(num_images, len(all_images)))

    # Class labels (adjust this based on your dataset)
    class_labels = list(train_generator.class_indices.keys())  # Use the same class indices as your training generator

    # Predict and display results
    for image_path in selected_images:
        class_index, confidence = predict(image_path)
        predicted_label = class_labels[class_index]
        print(f"Image: {image_path}")
        print(f"Predicted Class: {predicted_label}, Confidence: {confidence:.2f}")
        print("-" * 50)

# Path to test directory
test_dir = '/home/exh4748/ProjectTortoise/split_data/test'

# Predict random images
predict_random_images(test_dir, num_images=5)

1/1 [==============================] - 0s 92ms/step
Image: /home/exh4748/ProjectTortoise/split_data/test/efaz_augmented/20231112_204757_0_1951.jpg
Predicted Class: efaz_augmented, Confidence: 0.70
--------------------------------------------------
1/1 [==============================] - 0s 29ms/step
Image: /home/exh4748/ProjectTortoise/split_data/test/giannis_augmented/w640xh480_GettyImages-1238985680_0_2952.jpg
Predicted Class: giannis_augmented, Confidence: 1.00
--------------------------------------------------
1/1 [==============================] - 0s 27ms/step
Image: /home/exh4748/ProjectTortoise/split_data/test/ahmed/ahmed_0_3633.jpeg
Predicted Class: ahmed, Confidence: 0.85
--------------------------------------------------
1/1 [==============================] - 0s 28ms/step
Image: /home/exh4748/ProjectTortoise/split_data/test/efaz/efaz_0_7093.jpeg
Predicted Class: efaz, Confidence: 0.79
--------------------------------------------------
1/1 [==============================] - 0s 

# Flask Deployment

In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the trained model
model = load_model('facial_recognition_model.h5')

# Initialize the Flask app
app = Flask(__name__)

# Preprocessing function
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Match the input size of the model
    img_array = img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No image file provided'}), 400

    # Save the uploaded image
    image_file = request.files['image']
    image_path = './temp.jpg'
    image_file.save(image_path)

    # Preprocess the image
    processed_image = preprocess_image(image_path)

    # Make prediction
    predictions = model.predict(processed_image)
    class_index = np.argmax(predictions)
    confidence = np.max(predictions)

    # Map the class index to the label
    class_labels = list(train_generator.class_indices.keys())  # Adjust based on your class labels
    predicted_label = class_labels[class_index]

    return jsonify({
        'predicted_label': predicted_label,
        'confidence': float(confidence)
    })

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=5000)

# Testing API

In [ ]:
import requests

url = 'http://127.0.0.1:5000/predict'
# Replace with image_path
files = {'image': open(image_path, 'rb')}
response = requests.post(url, files=files)

print(response.json())